In [ ]:
print(1)

1


In [ ]:
import random

def generate_expression():
    operators = ['+', '-', '*', '/']
    expression = str(random.randint(1, 100))
    for _ in range(random.randint(0, 50)):
        operator = random.choice(operators)
        operand = str(random.randint(1, 100))
        expression += ' ' + operator + ' ' + operand
    return expression

def generate_dataset(num_samples=100000):
    data = []
    for _ in range(num_samples):
        expr = generate_expression()
        try:
            result = eval(expr)
            data.append((expr, result))
        except ZeroDivisionError:
            continue  # Skip expressions with division by zero
    return data

dataset = generate_dataset()

In [ ]:
import torch

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from collections import defaultdict
import numpy as np

# Create vocabulary
vocab = defaultdict(lambda: len(vocab))
for expr, _ in dataset:
    for token in expr.split():
        vocab[token]

vocab_size = len(vocab)
print(f'Vocabulary size: {vocab_size}')

# Convert expressions to sequences of integers
def tokenize(expr):
    return [vocab[token] for token in expr.split()]

# Pad sequences
def pad_sequence(seq, max_len):
    return seq + [0] * (max_len - len(seq))

max_seq_length = max(len(expr.split()) for expr, _ in dataset)
print(f'Max sequence length: {max_seq_length}')

# Prepare data
X = [tokenize(expr) for expr, _ in dataset]
X_padded = [pad_sequence(seq, max_seq_length) for seq in X]
y = [result for _, result in dataset]

X_padded = np.array(X_padded)
y = np.array(y)

Vocabulary size: 104
Max sequence length: 101


In [ ]:
import torch
import torch.nn as nn

class CalculatorLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(CalculatorLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        output = self.fc(lstm_out[:, -1, :])  # Take the last time step
        return output

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_padded, y, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.long).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
X_val_tensor = torch.tensor(X_val, dtype=torch.long).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).to(device)

In [ ]:
model = CalculatorLSTM(vocab_size=vocab_size, embedding_dim=100, hidden_dim=128, output_dim=1).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
batch_size = 32
num_epochs = 100

for epoch in range(num_epochs):
    for i in range(0, len(X_train), batch_size):
        inputs = X_train_tensor[i:i+batch_size]
        targets = y_train_tensor[i:i+batch_size]

        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/100, Loss: 393129492480.0
Epoch 2/100, Loss: 393129721856.0
Epoch 3/100, Loss: 393129918464.0
Epoch 4/100, Loss: 393130147840.0
Epoch 5/100, Loss: 393130278912.0
Epoch 6/100, Loss: 393130475520.0
Epoch 7/100, Loss: 393130541056.0
Epoch 8/100, Loss: 393130606592.0
Epoch 9/100, Loss: 393130541056.0
Epoch 10/100, Loss: 393130606592.0
Epoch 11/100, Loss: 393130672128.0
Epoch 12/100, Loss: 393130737664.0
Epoch 13/100, Loss: 393130475520.0
Epoch 14/100, Loss: 393130508288.0
Epoch 15/100, Loss: 393129426944.0
Epoch 16/100, Loss: 393126150144.0
Epoch 17/100, Loss: 393123463168.0
Epoch 18/100, Loss: 393120645120.0
Epoch 19/100, Loss: 393116844032.0
Epoch 20/100, Loss: 393114714112.0
Epoch 21/100, Loss: 393130180608.0
Epoch 22/100, Loss: 393116975104.0
Epoch 23/100, Loss: 393129066496.0
Epoch 24/100, Loss: 393107210240.0
Epoch 25/100, Loss: 393115664384.0
Epoch 26/100, Loss: 393141026816.0
Epoch 27/100, Loss: 393129689088.0
Epoch 28/100, Loss: 393100165120.0
Epoch 29/100, Loss: 393127100

What the fuck this loss

In [ ]:
model.eval()
with torch.no_grad():
    val_outputs = model(X_val_tensor.to(device))
    val_loss = criterion(val_outputs.squeeze(), y_val_tensor.to(device))
    print(f'Validation Loss: {val_loss.item()}')

OutOfMemoryError: CUDA out of memory. Tried to allocate 21.07 GiB. GPU 0 has a total capacity of 14.75 GiB of which 11.03 GiB is free. Process 38658 has 3.71 GiB memory in use. Of the allocated memory 3.37 GiB is allocated by PyTorch, and 212.31 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
def evaluate_expression(expr):
    tokens = tokenize(expr)
    padded = pad_sequence(tokens, max_seq_length)
    tensor = torch.tensor([padded], dtype=torch.long).to(device)
    model.eval()
    with torch.no_grad():
        output = model(tensor)
        result = output.item()
        print(f'Expression: {expr}')
        print(f'Predicted Result: {result}')
        print(f'Actual Result: {eval(expr)}')

# Test
test_expr = "3 + 5 * 2 - 4 / 2"
evaluate_expression(test_expr)

Expression: 2
Predicted Result: -622.423583984375
Actual Result: 2
